### Analysis for Bassez et al., 2021 (PMID: 33958794)

### Prerequisites

In [ ]:
#Load python
import anndata
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from scipy.stats import median_abs_deviation
import matplotlib.pyplot as plt
import os
import scvi
scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)
sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white', frameon=False)

### Load data

In [ ]:
Bassez2021 = sc.read_h5ad('../yourpath')
Bassez2021 = Bassez2021[Bassez2021.obs['anno'].isin(['Myeloid cells'])]
Bassez2021 = Bassez2021[Bassez2021.obs['expansion'].isin(['E', 'NE'])]
Bassez2021.obs['sample'] = Bassez2021.obs['patient_id']
Bassez2021.obs['response'] = Bassez2021.obs['expansion']
Bassez2021.layers["counts"] = Bassez2021.X.copy()
sc.pp.normalize_total(Bassez2021, target_sum=1e4)
sc.pp.log1p(Bassez2021)
Bassez2021.raw = Bassez2021
sc.pp.neighbors(Bassez2021, use_rep='X_scVI')
sc.tl.umap(Bassez2021)
sc.tl.leiden(Bassez2021, resolution=1)

### Annotate myeloid cells

In [ ]:
cell_type = {
    '0': 'mo-mac',
    '1': 'mo-mac',
    '2': 'mo-mac',
    '3': 'mo-mac',
    '4': 'mo-mac',
    '5': 'mo-mac', 
    '6': 'mo-mac',
    '7': 'mo-mac',
    '8': 'doublets_cancer_myeloid', 
    '9': 'mo-mac',
    '10': 'Monocytes',
    '11': 'DC2',
    '12': 'mo-mac_cycling',
    '13': 'DC2',
    '14': 'doublets_cancer_myeloid',
    '15': 'mo-mac',
    '16': 'mo-mac',
    '17': 'DC1',
    '18': 'mregDC',
}

Bassez2021.obs['anno_myeloid'] = Bassez2021.obs.leiden.map(cell_type)
Bassez2021 = Bassez2021[Bassez2021.obs['anno_myeloid'] != "doublets_cancer_myeloid"]

### ChEA-X analysis

In [ ]:
Bassez2021_sub = Bassez2021[Bassez2021.obs['timepoint'] == "On"]
Bassez2021v2_cluster_sub = Bassez2021_sub[Bassez2021_sub.obs['anno_myeloid'].isin(['mo-mac'])]
sc.tl.rank_genes_groups(Bassez2021v2_cluster_sub, 'response', method='wilcoxon')
results = Bassez2021v2_cluster_sub.uns['rank_genes_groups']
out = np.array([[0,0,0,0,0]])
for group in results['names'].dtype.names:
    out = np.vstack((out, np.vstack((results['names'][group],
                                     results['scores'][group],
                                     results['pvals_adj'][group],
                                     results['logfoldchanges'][group],
                                     np.array([group]*len(results['names'][group])).astype('object'))).T))
    
markers = pd.DataFrame(out[1:], columns = ['Gene', 'scores', 'pval_adj', 'lfc', 'cluster'])
markers = markers[(markers.pval_adj < 0.05) & (abs(markers.lfc) > 1)]
markers_up_NR = markers[markers['pval_adj'] <= 0.05]
markers_up_NR = markers_up_NR[markers_up_NR['lfc'] >= 0.5]
markers_up_NR = markers_up_NR[markers_up_NR['cluster'] == "NE"]
markers_up_NR = list(markers_up_NR['Gene'])

In [ ]:
import gseapy
names = gseapy.get_library_name()
print(names)

In [ ]:
enr_ChIPX_up = gseapy.enrichr(gene_list=markers_up_NR ,gene_sets=['ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X'],organism='Human', outdir='../yourpath/',cutoff=0.5)